In [ ]:
import xarray as xr

from src.tools import RiversHandler, GridHandler, OutputHandler

In [ ]:
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid_version3.nc')
river_ds = xr.open_dataset('fram_data/norfjords_160m_river_new.nc')

In [ ]:
rivhand = RiversHandler(grid_ds)
rivers = RiversHandler.check_rivers_entire_runoff(river_ds, runoff=30)

In [ ]:
rivers  # type: ignore

River 1: 12

In [ ]:
r_list_number = 1
rivhand.plot_rivers_luv(rivers[r_list_number-1:r_list_number])

In [ ]:
riv_number = 12
RiversHandler.check_coords(river_ds, 728, 11)

In [ ]:
rivhand.plot_rivers_luv([[-1, 728, 10, 0, -1]])

In [ ]:
river_ds = rivhand.split_report(river_ds, riv_number, 
                                dir_swap=True, flux_swap=True, 
                                f_xi=728, f_eta=10,
                                s_xi=728, s_eta=9)